In [2]:
import sqlite3
database_name = 'fetch_data.db'  
conn = sqlite3.connect(database_name)
cursor = conn.cursor()

### Check receipt_items table

In [24]:
# Execute a query to Check for missing values, especially in key fields like barcode, brand_id, quantity_purchased, item_price, and final_price 
## in receipt_items.csv
cursor.execute('SELECT * FROM receipt_items where barcode is null')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:10]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))


cursor.execute('SELECT COUNT(*) FROM receipt_items where barcode is null')
count = cursor.fetchone()[0]
print()
print(f"There are {count} rows with null barcode")

receipt_item_id | user_id | barcode | receipt_id | brand_id | brand_code | description | quantity_purchased | item_price | final_price | needs_fetch_review | partner_item_id | prevent_target_gap_points | user_flagged_barcode | user_flagged_new_item | user_flagged_price | user_flagged_quantity
--------------------------------------------------
4 | 5ff1e1f1cfcf6c399c274b0b | NULL | 5ff1e1f10a720f052300057a | NULL | NULL | NULL | NULL | NULL | NULL | 0 | 1 | 1 | 4011 | 1 | 26.0 | 3
9 | 5ff1e194b6a9d73a3a9f1052 | NULL | 5ff1e1cd0a720f052300056f | NULL | MISSION | MSSN TORTLLA | 1 | 2.23 | 2.23 | NULL | 1009 | NULL | NULL | NULL | NULL | NULL
26 | 5f9c74f7c88c1415cbddb839 | NULL | 5f9c74f70a7214ad07000037 | NULL | NULL | NULL | NULL | NULL | NULL | 1 | 2 | 1 | 034100573065 | 1 | 29.0 | 1
27 | 5f9c74f7c88c1415cbddb839 | NULL | 5f9c74f70a7214ad07000037 | NULL | NULL | NULL | NULL | NULL | NULL | 1 | 3 | 1 | 034100573065 | 1 | 29.0 | 1
28 | 5f9c74f7c88c1415cbddb839 | NULL | 5f9c74f70a7214ad070

barcode is foreign key to brands table but it has many NULL values. Similar to other fields such as brand_id, quantity_purchased, item_price, and final_price.

In [4]:
# Execute a query to see rows with Data Inconsistency: Ensure that item_price and final_price match for each item.
## in receipt_items.csv
cursor.execute('SELECT * FROM receipt_items where item_price != final_price')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

receipt_item_id | user_id | barcode | receipt_id | brand_id | brand_code | description | quantity_purchased | item_price | final_price | needs_fetch_review | partner_item_id | prevent_target_gap_points | user_flagged_barcode | user_flagged_new_item | user_flagged_price | user_flagged_quantity
--------------------------------------------------
1826 | 60025fe0e257124ec6b99a87 | NULL | 600260210a720f05f300008f | NULL | NULL | HYV SWT SMKD THICK | 1 | 4.99 | 2.88 | NULL | 1213 | NULL | NULL | NULL | NULL | NULL
2265 | 60049cdce257124ec6b9a353 | NULL | 60049d9d0a720f05f3000094 | NULL | DARE | DARE CRACKER | 1 | 3.59 | 2.89 | NULL | 1371 | NULL | NULL | NULL | NULL | NULL
2267 | 60049cdce257124ec6b9a353 | NULL | 60049d9d0a720f05f3000094 | NULL | NULL | Cookies | 1 | 2.59 | 2.39 | NULL | 1374 | NULL | NULL | NULL | NULL | NULL
2268 | 60049cdce257124ec6b9a353 | NULL | 60049d9d0a720f05f3000094 | NULL | LINDT | LINDT CHOCOLATE | 1 | 2.99 | 2.5 | NULL | 1376 | NULL | NULL | NULL | NULL | NULL


Item price and final_price are not matched in some cases.

In [5]:
# Execute a query to see rows Invalid Values: item_price is zero or negative.
## in receipt_items.csv
cursor.execute('SELECT * FROM receipt_items where item_price <= 0')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

receipt_item_id | user_id | barcode | receipt_id | brand_id | brand_code | description | quantity_purchased | item_price | final_price | needs_fetch_review | partner_item_id | prevent_target_gap_points | user_flagged_barcode | user_flagged_new_item | user_flagged_price | user_flagged_quantity
--------------------------------------------------
5044 | 600f20c15edb787dce060911 | NULL | 600f24970a720f053500002f | NULL | NULL | KINDER JOY E 1 50 F | 1 | 0.0 | 0.0 | NULL | 1646 | NULL | NULL | NULL | NULL | NULL
5293 | 600f00d05edb787dce05fb84 | NULL | 600f0cc70a720f053500002c | NULL | NULL | STACARR MRKTP C | 1 | 0.0 | 0.0 | NULL | 1323 | NULL | NULL | NULL | NULL | NULL
5864 | 600f35015edb782098e2ac1b | NULL | 600f39c30a7214ada2000030 | NULL | NULL | KINDER JOY E 1 50 F | 1 | 0.0 | 0.0 | NULL | 1631 | NULL | NULL | NULL | NULL | NULL
6501 | 600f29a64329897eac239049 | NULL | 600f2fc80a720f0535000030 | NULL | NULL | KINDER JOY E 1.50 F | 1 | 0.0 | 0.0 | 0 | 1653 | NULL | NULL | NULL | NULL |

Some items have zero in item_price and final_price

### Check Receipts Table

In [26]:
## Missing Values: Check for missing values in critical fields such as purchase_date and total_spent in receipts table
cursor.execute('SELECT * FROM receipts where purchase_date is null or total_spent is null')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:20]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

cursor.execute('SELECT COUNT(*) FROM receipts where purchase_date is null or total_spent is null')
count = cursor.fetchone()[0]
print()
print(f"There are {count} rows with null purchase_date and total_spent")

receipt_id | purchase_date | create_date | date_scanned | finished_date | modify_date | points_awarded_date | total_spent | bonus_points_earned | bonus_points_earned_reason | purchased_item_count | rewards_receipt_status
--------------------------------------------------
5ff1e1e90a7214ada1000569 | NULL | 2021-01-03T07:25:29 | 2021-01-03T07:25:29 | NULL | 2021-01-03T07:25:29 | NULL | 0.0 | NULL | NULL | 0 | FLAGGED
5ff475820a7214ada10005cf | NULL | 2021-01-05T06:19:46 | 2021-01-05T06:19:46 | NULL | 2021-01-05T06:19:46 | NULL | NULL | NULL | NULL | NULL | SUBMITTED
5ff4ce3c0a720f05230005c4 | NULL | 2021-01-05T12:38:20 | 2021-01-05T12:38:20 | NULL | 2021-01-05T12:38:20 | NULL | 0.0 | NULL | NULL | 0 | FLAGGED
5ff5ecb90a7214ada10005f9 | NULL | 2021-01-06T09:00:40 | 2021-01-06T09:00:40 | NULL | 2021-01-06T09:00:40 | NULL | NULL | NULL | NULL | NULL | SUBMITTED
5ff73be90a720f052300060a | NULL | 2021-01-07T08:50:49 | 2021-01-07T08:50:49 | NULL | 2021-01-07T08:50:49 | NULL | 0.0 | NULL | NULL 

There are 448 missing entries in purchase date or total_spent in receipts table

In [12]:
## Check uniqueness of receipt_id: receipts table
cursor.execute('SELECT count(distinct receipt_id) - count(*) FROM receipts')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:20]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

count(distinct receipt_id) - count(*)
--------------------------------------------------
0


In [27]:
## Check Date Inconsistencies: 
cursor.execute('SELECT * FROM receipts WHERE purchase_date > finished_date')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:20]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

receipt_id | purchase_date | create_date | date_scanned | finished_date | modify_date | points_awarded_date | total_spent | bonus_points_earned | bonus_points_earned_reason | purchased_item_count | rewards_receipt_status
--------------------------------------------------
5ff1e1b20a7214ada100055a | 2021-02-03T07:24:35 | 2021-01-03T07:24:34 | 2021-01-03T07:24:34 | 2021-01-03T07:24:35 | 2021-01-03T07:24:35 | 2021-01-03T07:24:35 | 1.0 | 300 | Receipt number 4 completed, bonus point schedule DEFAULT (5cefdcacf3693e0b50e83a36) | 1 | FINISHED
5ff4ce640a7214ada10005e0 | 2021-02-05T12:39:00 | 2021-01-05T12:39:00 | 2021-01-05T12:39:00 | 2021-01-05T12:39:00 | 2021-01-05T12:39:00 | 2021-01-05T12:39:00 | 1.0 | 25 | COMPLETE_NONPARTNER_RECEIPT | 1 | FINISHED
5fff26ee0a720f05f300001a | 2021-02-13T08:59:26 | 2021-01-13T08:59:26 | 2021-01-13T08:59:26 | 2021-01-13T08:59:26 | 2021-01-13T08:59:26 | 2021-01-13T08:59:26 | 1.0 | 25 | COMPLETE_NONPARTNER_RECEIPT | 1 | FINISHED
600887560a720f05fa000098 | 2021-

7 cases where purchase_date is later than finished_date

### Check brands table

In [19]:
## Check uniqueness of brand_id, barcode from brands table
cursor.execute('SELECT barcode FROM brands GROUP BY 1 HAVING COUNT(*) > 1')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:20]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

## Check uniqueness of brand_id, barcode from brands table
cursor.execute('SELECT brand_id FROM brands GROUP BY 1 HAVING COUNT(*) > 1')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:20]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

barcode
--------------------------------------------------
511111004790
511111204923
511111305125
511111504139
511111504788
511111605058
511111704140
brand_id
--------------------------------------------------


There are some duplicates in barcode. All brand_id are unique, which is the PK of the table.

In [29]:
## Check missing values of brand_code in brands table
cursor.execute('SELECT COUNT(*) FROM brands WHERE brand_code IS NULL')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:20]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

COUNT(*)
--------------------------------------------------
234


There are 269 missing value in brand_code field.

### Check users table

In [21]:
## Check uniqueness of brand_id, barcode from brands table
cursor.execute("SELECT * FROM users where role != 'consumer' ")
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(f"{' | '.join(column_names)}")
print('-' * 50)
for row in rows[:20]:
    print(' | '.join(str(item) if item is not None else 'NULL' for item in row))

user_id | state | created_date | last_login | role | active
--------------------------------------------------
59c124bae4b0299e55b0f330 | WI | 2017-09-19 07:07:54.302000 | 2021-02-08 08:42:58.117000 | fetch-staff | 1
5f2068904928021530f8fc34 | WI | 2020-07-28 11:04:00.905000 | 2021-02-04 07:30:05.375000 | fetch-staff | 1
5fbc35711d967d1222cbfefc | NULL | 2020-11-23 14:19:29.509000 | 2021-02-25 20:25:51.057000 | fetch-staff | 1
5fc961c3b8cfca11a077dd33 | NH | 2020-12-03 14:08:03.936000 | 2021-02-26 14:39:16.799000 | fetch-staff | 1
5fa41775898c7a11a6bcef3e | NULL | 2020-11-05 07:17:09.396000 | 2021-03-04 08:02:02.026000 | fetch-staff | 1
5fa32b4d898c7a11a6bcebce | AL | 2020-11-04 14:29:33.309000 | 2021-03-03 23:21:58.047000 | fetch-staff | 1
5964eb07e4b03efd0c0f267b | IL | 2017-07-11 08:13:11.771000 | 2021-03-04 11:07:49.770000 | fetch-staff | 1
54943462e4b07e684157a532 | NULL | 2014-12-19 06:21:22.381000 | 2021-03-05 08:52:23.204000 | fetch-staff | 1


There are 8 cases where role column is not set to CONSUMER as constant value. 